In [68]:
import random as rand
from operator import attrgetter
from sortedcontainers import SortedList
import bisect
import math
import random
from pyskiplist import SkipList

In [53]:
def binarysearch(a, x):
    i = bisect.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return True
    else:
        return False

In [222]:
class MaxList:
    def __init__(self):
        self.nodes = []
        self.max = float("-inf")

    def __lt__(self, other):
        if isinstance(other, int):
            return self.max < other
        else:
            return self.max < other.max

class MinMaxList:
    def __init__(self):
        self.sublists = []
        self.min = float("inf")
        self.max = float("-inf")

    def __lt__(self, other):
        return self.min < other

class TeleportList:

    def __init__(self):
        self.height = -1
        self.blists = []
        self.load = 999

    def lookup(self, nr):
        ## Search MinMaxLists in TeleportList
        for he in self.blists:
            if nr < he.min:
                break
            else:
                if he.max >= nr:
                    ## Search MaxLists in MinMaxList
                    for sl in he.sublists:
                        if nr < sl.max:
                            break
                        else:
                            if binarysearch(sl.nodes, nr):
                                return True
        return False

    def insert(self, nr):
        ## Getting the estimated geometric distribution
        height = int(-(math.log2(random.random())))
        ## Checking whether we need to add new edges
        if self.height < height:
            for i in range(height - self.height):
                self.blists.append(MinMaxList())
                self.blists[self.height + i].sublists.append(MaxList())
            self.height = height

        ## Getting the to-be-added list
        blist = self.blists[height-1]

        ## Doing the search to see which MaxList it should be in
        L = len(blist.sublists)
        i = bisect.bisect_left(blist.sublists, nr)
        ## If it's there don't insert
        if i != L and blist.sublists[i] == nr:
            return False
        else:
            ## If it's smaller than all other elements there then just insort it
            if i == 0:
                new_max = MaxList()
                new_max.nodes.append(nr)
                new_max.max = nr
                #bisect.insort_left(blist.sublists, new_max)
                blist.sublists.append(new_max)
            ## If it's bigger than just append to end
            elif i == L:
                blist.sublists[max(i - 1, 0)].nodes.append(nr)
                if nr > blist.sublists[max(i - 1, 0)].max:
                    blist.sublists[max(i - 1, 0)].max = nr
                #blist.sublists.sort(key=attrgetter('max'))
            ## Else add it
            else:
                candidate_sublist = blist.sublists[i]
                ## If MaxList has under 1000 elements then just insort
                if len(candidate_sublist.nodes) <= self.load:
                    bisect.insort_left(candidate_sublist.nodes, nr)
                ## Else create a new MaxList and insort it there
                else:
                    new_max = MaxList()
                    new_max.nodes.append(nr)
                    new_max.max = nr
                    blist.sublists.append(new_max)
                    #bisect.insort_left(blist.sublists, new_max)
            ## Update MinMax
            if blist.min > nr:
                blist.min = nr
            if blist.max < nr:
                blist.max = nr
            blist.sublists.sort(reverse=True, key=attrgetter('max'))
            self.blists.sort(key=attrgetter('min'))

    def show_hedges(self):
        for i in self.blists:
            maxes = [j.max for j in i.sublists]
            print(maxes)

    def show_minmax(self):
        for i in self.blists:
            print(f'({i.min}, {i.max})')

In [224]:
hgraph = TeleportList()
skeepFast = SkipList()
slist = SortedList()

nr = 100000

ten_thousand_integers = [rand.randint(1, 1000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.insert(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        sl.add(ten_thousand_integers[i])

def ten_k_skeep(skeep):
    for i in range(nr):
        skeep.insert(ten_thousand_integers[i], ten_thousand_integers[i])

%timeit -r 1 -n 1 ten_k_hg(hgraph)
#%timeit -r 1 -n 1 ten_k_sl(slist)
#%timeit -r 1 -n 1 ten_k_skeep(skeepFast)

1min 10s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [235]:
def ten_k_hg(HG):
    for i in range(nr):
        HG.lookup(ten_thousand_integers[i])

def ten_k_sl(sl):
    for i in range(nr):
        ten_thousand_integers[i] in sl

def ten_k_skeep(skeep):
    for i in range(nr):
        ten_thousand_integers[i] in skeep

%timeit -r 1 -n 1 ten_k_hg(hgraph)
#%timeit -r 10 -n 1 ten_k_sl(slist)
#%timeit -r 10 -n 1 ten_k_skeep(skeepFast)

544 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [236]:
hgraph.show_minmax()
hgraph.show_hedges()

(6, 999968)
(7, 999860)
(38, 999869)
(47, 999482)
(50, 999967)
(102, 999594)
(102, 999610)
(115, 999770)
(135, 999692)
(140, 999972)
(157, 996956)
(162, 997817)
(163, 999843)
(178, 999895)
(188, 998799)
(254, 996178)
(304, 999820)
(323, 998252)
(338, 998641)
(357, 999620)
(362, 999924)
(371, 998050)
[999968, 999930, 999752, 999667, 999623, 999576, 999502, 999502, 999431, 999431, 999326, 999252, 999233, 999228, 999216, 999168, 999140, 999114, 999044, 998988, 998968, 998928, 998883, 998839, 998808, 998800, 998736, 998698, 998694, 998670, 998616, 998559, 998500, 998368, 998357, 998356, 998338, 998321, 998281, 998177, 998164, 998146, 998117, 998071, 998065, 998046, 998005, 997946, 997881, 997879, 997873, 997852, 997825, 997825, 997715, 997653, 997627, 997585, 997455, 997424, 997370, 997369, 997334, 997275, 997259, 997180, 997140, 997103, 997057, 997036, 997028, 996953, 996911, 996878, 996788, 996777, 996774, 996760, 996741, 996740, 996577, 996556, 996451, 996417, 996416, 996361, 996353, 99